Read in `inputs.yml` and produce a GraphViz representation of the tables

In [57]:
import os
import yaml

import cea.scripts
import cea.interfaces.dashboard.inputs

from jinja2 import Template

import cea.inputlocator
import cea.config

config = cea.config.Configuration()
locator = cea.inputlocator.InputLocator(scenario=config.scenario)
schemas = cea.scripts.schemas()

In [58]:
inputs_yml = os.path.abspath(os.path.join(os.path.dirname(cea.interfaces.dashboard.inputs.__file__), "inputs.yml"))

with open(inputs_yml, 'r') as fp:
    inputs = yaml.load(fp)

inputs

{'air-conditioning-systems': {'fields': [{'name': 'Name', 'type': 'str'},
   {'choice_properties': {'lookup': {'column': 'code',
      'path': 'get_database_air_conditioning_systems',
      'sheet': 'COOLING'}},
    'name': 'type_cs',
    'type': 'choice'},
   {'choice_properties': {'lookup': {'column': 'code',
      'path': 'get_database_air_conditioning_systems',
      'sheet': 'HEATING'}},
    'name': 'type_hs',
    'type': 'choice'},
   {'choice_properties': {'lookup': {'column': 'code',
      'path': 'get_database_air_conditioning_systems',
      'sheet': 'HOT_WATER'}},
    'name': 'type_dhw',
    'type': 'choice'},
   {'choice_properties': {'lookup': {'column': 'code',
      'path': 'get_database_air_conditioning_systems',
      'sheet': 'CONTROLLER'}},
    'name': 'type_ctrl',
    'type': 'choice'},
   {'choice_properties': {'lookup': {'column': 'code',
      'path': 'get_database_air_conditioning_systems',
      'sheet': 'VENTILATION'}},
    'name': 'type_vent',
    'type': 'ch

In [59]:
table_template = """
{{locator}} [shape=none, margin=0, label=<
    <table border="0" cellborder="1" cellspacing="0" cellpadding="4" >
        <tr><td bgcolor="lightblue">{{table}}</td></tr>
        {% for field in fields %}<tr><td port="{{field.name}}" align="left">{{field.type}}: {{field.name}}</td></tr>
        {% endfor %}
    </table>>];
"""

In [60]:
table_defs = []

for table in inputs.keys():
    lm = inputs[table]["location"]
    fields = inputs[table]["fields"]
    table_defs.append(Template(table_template).render(locator=lm, table=table, fields=fields))

In [61]:
print "\n".join(table_defs)


get_building_air_conditioning [shape=none, margin=0, label=<
    <table border="0" cellborder="1" cellspacing="0" cellpadding="4" >
        <tr><td bgcolor="lightblue">air-conditioning-systems</td></tr>
        <tr><td port="Name" align="left">str: Name</td></tr>
        <tr><td port="type_cs" align="left">choice: type_cs</td></tr>
        <tr><td port="type_hs" align="left">choice: type_hs</td></tr>
        <tr><td port="type_dhw" align="left">choice: type_dhw</td></tr>
        <tr><td port="type_ctrl" align="left">choice: type_ctrl</td></tr>
        <tr><td port="type_vent" align="left">choice: type_vent</td></tr>
        <tr><td port="heat_starts" align="left">date: heat_starts</td></tr>
        <tr><td port="heat_ends" align="left">date: heat_ends</td></tr>
        <tr><td port="cool_starts" align="left">date: cool_starts</td></tr>
        <tr><td port="cool_ends" align="left">date: cool_ends</td></tr>
        
    </table>>];

get_zone_geometry [shape=none, margin=0, label=<
    

In [62]:
databases = set()  # (lm, sheet)
connections = set()  # (ltable, lfield, rtable, sheet, rfield)

for ltable in inputs:
    for field in inputs[ltable]["fields"]:
        if field["type"] == "choice":
            lfield = field["name"]
            rtable = field["choice_properties"]["lookup"]["path"]
            sheet = field["choice_properties"]["lookup"]["sheet"]
            rfield = field["choice_properties"]["lookup"]["column"]
            databases.add((rtable, sheet))
            connections.add((ltable, lfield, rtable, sheet, rfield))

databases

{('get_database_air_conditioning_systems', 'CONTROLLER'),
 ('get_database_air_conditioning_systems', 'COOLING'),
 ('get_database_air_conditioning_systems', 'HEATING'),
 ('get_database_air_conditioning_systems', 'HOT_WATER'),
 ('get_database_air_conditioning_systems', 'VENTILATION'),
 ('get_database_construction_standards', 'STANDARD_DEFINITION'),
 ('get_database_envelope_systems', 'CONSTRUCTION'),
 ('get_database_envelope_systems', 'FLOOR'),
 ('get_database_envelope_systems', 'ROOF'),
 ('get_database_envelope_systems', 'SHADING'),
 ('get_database_envelope_systems', 'TIGHTNESS'),
 ('get_database_envelope_systems', 'WALL'),
 ('get_database_envelope_systems', 'WINDOW'),
 ('get_database_supply_assemblies', 'COOLING'),
 ('get_database_supply_assemblies', 'ELECTRICITY'),
 ('get_database_supply_assemblies', 'HEATING'),
 ('get_database_supply_assemblies', 'HOT_WATER'),
 ('get_database_use_types_properties', 'INTERNAL_LOADS')}

In [73]:
connections

{('air-conditioning-systems',
  'type_cs',
  'get_database_air_conditioning_systems',
  'COOLING',
  'code'),
 ('air-conditioning-systems',
  'type_ctrl',
  'get_database_air_conditioning_systems',
  'CONTROLLER',
  'code'),
 ('air-conditioning-systems',
  'type_dhw',
  'get_database_air_conditioning_systems',
  'HOT_WATER',
  'code'),
 ('air-conditioning-systems',
  'type_hs',
  'get_database_air_conditioning_systems',
  'HEATING',
  'code'),
 ('air-conditioning-systems',
  'type_vent',
  'get_database_air_conditioning_systems',
  'VENTILATION',
  'code'),
 ('architecture',
  'type_base',
  'get_database_envelope_systems',
  'FLOOR',
  'code'),
 ('architecture',
  'type_cons',
  'get_database_envelope_systems',
  'CONSTRUCTION',
  'code'),
 ('architecture',
  'type_floor',
  'get_database_envelope_systems',
  'FLOOR',
  'code'),
 ('architecture',
  'type_leak',
  'get_database_envelope_systems',
  'TIGHTNESS',
  'code'),
 ('architecture',
  'type_roof',
  'get_database_envelope_system

In [72]:
db_template = """
{{locator}}_{{sheet}} [shape=none, margin=0, label=<
    <table border="0" cellborder="1" cellspacing="0" cellpadding="4" >
        <tr><td bgcolor="lightgrey">{{table}}</td></tr>
        {% for field in fields %}<tr><td port="{{field.name}}" align="left">{{field.type}}: {{field.name}}</td></tr>
        {% endfor %}
    </table>>];
"""

db_defs = []

for lm, sheet in databases:
    xlsx = os.path.basename(getattr(locator, lm)())
    table = "{file}:{sheet}".format(file=xlsx, sheet=sheet)
    fields = schemas[lm]["schema"][sheet]
    Template(db_template).render(locator=lm, sheet=sheet, table=table, fields=field)

KeyError: 'get_database_use_types_properties'

In [70]:
schemas["get_database_envelope_systems"]["schema"]["ROOF"]

{'Description': {'sample_data': 'concrete or rock pebbles finishing - Singapore',
  'types_found': ['string']},
 'GHG_ROOF_kgCO2m2': {'sample_data': 100, 'types_found': ['float']},
 'U_roof': {'sample_data': 0.6, 'types_found': ['float']},
 'a_roof': {'sample_data': 0.6, 'types_found': ['float']},
 'code': {'sample_data': 'T7', 'types_found': ['string']},
 'e_roof': {'sample_data': 0.94, 'types_found': ['float']},
 'r_roof': {'sample_data': 0.4, 'types_found': ['float']}}